In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pd.set_option('display.max_columns', 500)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings('ignore')

In [ ]:
test=pd.read_csv('../input/jobathon-analytics-vidhya-health-insurance/Test.csv')
test.shape

In [ ]:
df=pd.read_csv('../input/jobathon-analytics-vidhya-health-insurance/Train.csv')
df.shape

In [ ]:
t=pd.read_csv('../input/jobathon-analytics-vidhya-health-insurance/Train.csv')
t.shape

In [ ]:
df.dtypes

In [ ]:
columns=['Accomodation_Type','Reco_Insurance_Type','Is_Spouse','Holding_Policy_Type',
          'Health Indicator','Holding_Policy_Duration','Reco_Policy_Cat']
for col in columns:
    df[col]=df[col].astype('category')

In [ ]:
def barplot(column,df):
    bar_plot1 = sns.countplot(x=column,data=df)

    total = len(df[column])
    for p in bar_plot1.patches:
        percentage = '{:.2f}%'.format(100 * p.get_height()/total)
        height = p.get_height()
        bar_plot1.text(p.get_x() + p.get_width()/2, height, percentage,ha='center')

In [ ]:
barplot('Accomodation_Type',df)

In [ ]:
barplot('Reco_Insurance_Type',df)

In [ ]:
barplot('Is_Spouse',df)

In [ ]:
barplot('Response',df)

In [ ]:
plt.subplots(figsize=(10,5))
ax = df['Health Indicator'].value_counts().plot.bar(width=0.9, color=sns.color_palette('bright',20), fontsize=8)
for p in ax.patches:
    ax.annotate(format(p.get_height()+0.5), (p.get_x()+0.15,p.get_height()+1))
plt.title('Health Indicator', fontsize=15)
plt.xlabel('Health', fontsize=15)
plt.ylabel('count',fontsize=15)
plt.show()

In [ ]:
plt.subplots(figsize=(10,5))
ax = df['Holding_Policy_Duration'].value_counts().plot.bar(width=0.9, color=sns.color_palette('bright',20), fontsize=8)
for p in ax.patches:
    ax.annotate(format(p.get_height()+0.5), (p.get_x()+0.15,p.get_height()+1))
plt.title('Holding_Policy_Duration', fontsize=15)
plt.xlabel('Duration', fontsize=15)
plt.ylabel('count',fontsize=15)
plt.show()

In [ ]:
plt.subplots(figsize=(10,5))
ax = df['Reco_Policy_Cat'].value_counts().plot.bar(width=0.9, color=sns.color_palette('bright',20), fontsize=8)
for p in ax.patches:
    ax.annotate(format(p.get_height()+0.5), (p.get_x()+0.15,p.get_height()+1))
plt.title('Reco_Policy_Cat', fontsize=15)
plt.xlabel('Reco_Policy', fontsize=15)
plt.ylabel('count',fontsize=15)
plt.show()

In [ ]:
import plotly
import plotly.graph_objs as go

In [ ]:
trace1 = go.Box(y = df.Upper_Age[df.Response==0], name='Not Responded')
trace2 = go.Box(y=df.Upper_Age[df.Response==1], name='Responded')

data = [trace1,trace2]
layout = go.Layout(width=600,height=400,title='Response by upper age')
fig = go.Figure(data=data, layout=layout)
fig.show()

In [ ]:
trace1 = go.Box(y = df.Reco_Policy_Premium[df.Response==0], name='Not Responded')
trace2 = go.Box(y=df.Reco_Policy_Premium[df.Response==1], name='Responded')

data = [trace1,trace2]
layout = go.Layout(width=600,height=400,title='Response by Reco_Policy_Premium')
fig = go.Figure(data=data, layout=layout)
fig.show()

In [ ]:
df.Reco_Insurance_Type.value_counts()

In [ ]:
temp = df.groupby(['Reco_Insurance_Type','Response']).size().to_frame()
temp = temp.reset_index()
temp.columns = ['Reco_Insurance_Type','Response','Count']



trace1 = go.Bar(x=temp.Response[temp.Reco_Insurance_Type=='Individual'],
               y=temp.Count[temp.Reco_Insurance_Type=='Individual']/temp.Count.sum(), name='Individual')
trace2 = go.Bar(x=temp.Response[temp.Reco_Insurance_Type=='Joint'],
               y=temp.Count[temp.Reco_Insurance_Type=='Joint']/temp.Count.sum(), name='Joint')

data = [trace1,trace2]
layout = go.Layout(width=800, height=500, title='Response by joint vs individual', barmode='stack')
fig = go.Figure(data=data, layout=layout)
fig.show()

In [ ]:
temp = df.groupby(['Holding_Policy_Type','Response']).size().to_frame()
temp = temp.reset_index()
temp.columns = ['Holding_Policy_Type','Response','Count']



trace1 = go.Bar(x=temp.Response[temp.Holding_Policy_Type==1.0],
               y=temp.Count[temp.Holding_Policy_Type==1.0]/temp.Count.sum())
trace2 = go.Bar(x=temp.Response[temp.Holding_Policy_Type==2.0],
               y=temp.Count[temp.Holding_Policy_Type==2.0]/temp.Count.sum())
trace3 = go.Bar(x=temp.Response[temp.Holding_Policy_Type==3.0],
               y=temp.Count[temp.Holding_Policy_Type==3.0]/temp.Count.sum())
trace4 = go.Bar(x=temp.Response[temp.Holding_Policy_Type==4.0],
               y=temp.Count[temp.Holding_Policy_Type==4.0]/temp.Count.sum())

data = [trace1,trace2,trace3,trace4]
layout = go.Layout(width=800, height=500, title='Response by joint vs Holding_Policy_Type', barmode='stack')
fig = go.Figure(data=data, layout=layout)
fig.show()

In [ ]:
sns.distplot(df['Upper_Age'], bins=100)

In [ ]:
sns.distplot(df['Lower_Age'], bins=100)

In [ ]:
bins=0,5,10,15,20,25
Reco_Policy_bin = ['one','two','three','four','five']
df['recommended_insurance_bin']=pd.cut(df['Reco_Policy_Cat'], bins, labels=Reco_Policy_bin)

In [ ]:
df.drop('Reco_Policy_Cat', axis=1,inplace=True)

In [ ]:
from scipy.stats import chi2_contingency
from scipy.stats import chi2

## conda install -c anaconda statsmodels
import statsmodels.api as sm

In [ ]:
cat_col = t.select_dtypes(['category']).columns

In [ ]:
def find_chi2_independence(cat_col, target, t, alpha=0.05):
    data = t[[target, cat_col]]
    tab = sm.stats.Table.from_data(data)
    tab = tab.table_orig.to_numpy()
    print(f"---------------{target} Vs {cat_col} Chi Square Test of Independence -------------------")
    print(f"\n Contingency table :\n")
    print(tab)
    
    stat, p, dof, expected = chi2_contingency(tab)
    print(f"\n Expected table :\n")
    print(expected)
    
    print(f"The p value returned = {p} and degrees of freedom returned = {dof}")
    
    # interpret p-value
    print('significance(alpha) = %.3f' % (alpha))

    if p <= alpha:
        print('Dependent (reject H0)')
    else:
        print('Independent (fail to reject H0)') 

In [ ]:
for col in cat_col:
    find_chi2_independence(col, 'Response', t)

In [ ]:
df.drop(['ID','City_Code','Region_Code'], axis=1,inplace=True)

In [ ]:
y=df['Response']
X=df.drop('Response', axis=1)
X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.3,random_state=101)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

In [ ]:
num=X_train.select_dtypes(['int64','float64']).columns
cat=X_train.select_dtypes('category').columns

In [ ]:
impute = SimpleImputer(strategy='most_frequent')
impute.fit(X_train[cat])
X_train[cat] = impute.transform(X_train[cat])
X_val[cat] = impute.transform(X_val[cat])

In [ ]:
scaler=StandardScaler()
scaler.fit(X_train[num])
X_train_std = scaler.transform(X_train[num])
X_val_std = scaler.transform(X_val[num])
X_train_num = pd.DataFrame(X_train_std, columns=num)
X_val_num = pd.DataFrame(X_val_std, columns=num)

In [ ]:
ohe=OneHotEncoder()
ohe.fit(X_train[cat])

ohe_cols = list(ohe.get_feature_names(cat))

X_train_ohe = ohe.transform(X_train[cat])
X_val_ohe = ohe.transform(X_val[cat])
X_train_cat=pd.DataFrame(X_train_ohe.todense(), columns=ohe_cols)
X_val_cat=pd.DataFrame(X_val_ohe.todense(), columns=ohe_cols)

X_train = pd.concat([X_train_num,X_train_cat],axis=1)
X_val = pd.concat([X_val_num,X_val_cat],axis=1)

In [ ]:
scores = pd.DataFrame(columns=['Model','Train_roc_auc','val_roc_auc'])

def get_metrics(train_actual,train_predicted,val_actual,val_predicted,model_description,dataframe):
    train_roc_auc = roc_auc_score(train_actual,train_predicted)
  
    val_roc_auc = roc_auc_score(val_actual,val_predicted)

    dataframe = dataframe.append(pd.Series([model_description, train_roc_auc,val_roc_auc],
                                           index=scores.columns ), ignore_index=True)
    return(dataframe)

In [ ]:
clf=DecisionTreeClassifier(max_depth=4)
clf.fit(X_train, y_train)
train_pred = clf.predict(X_train)
test_pred = clf.predict(X_val)

tree_train=roc_auc_score(y_train, train_pred)
tree_test=roc_auc_score(y_val, test_pred)
print(tree_train)
print(tree_test)

In [ ]:
scores = get_metrics(y_train,train_pred,y_val,test_pred,"Decision Tree_NORMAL",scores)
scores

In [ ]:
param_grid = {"class_weight":['balanced', None],
              "criterion": ["gini", "entropy"],
              "max_depth": [2,3,4,5],
              "min_samples_leaf": [2,3,4,5,7],
               "max_leaf_nodes": [None, 5, 10, 12, 14]
              }
dt = DecisionTreeClassifier(random_state=123)
clf_dt_grid = GridSearchCV(dt, param_grid,cv=5,scoring='accuracy')
clf_dt_grid.fit(X_train, y_train)
grid_train=clf_dt_grid.predict(X_train)
grid_test=clf_dt_grid.predict(X_val)

tree_train=roc_auc_score(y_train, grid_train)
tree_test=roc_auc_score(y_val, grid_test)
print(tree_train)
print(tree_test)

In [ ]:
clf_dt_grid.best_params_

In [ ]:
scores = get_metrics(y_train,grid_train,y_val,grid_test,"Decision Tree_grid",scores)
scores

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, truncnorm, randint

clf_dt = DecisionTreeClassifier(random_state=123, class_weight='balanced')

In [ ]:
max_leaf_nodes = np.random.normal(loc=5, scale=1, size=5).astype(int)


max_depth = np.random.uniform(1,10,4).astype(int)

min_samples_split = np.random.uniform(2,10, 5).astype(int)

model_params = {
    'max_depth': list(max_depth),
    'max_leaf_nodes': list(max_leaf_nodes),
    'min_samples_split': list(min_samples_split)
}

clf_random = RandomizedSearchCV(estimator=clf_dt, param_distributions=model_params, n_iter=300, cv=5, scoring='accuracy', n_jobs=-1)
clf_random.fit(X_train, y_train)

clf_dt_random = clf_random.best_estimator_
print(clf_dt_random)

y_pred_train = clf_dt_random.predict(X_train)
y_pred_val = clf_dt_random.predict(X_val)

In [ ]:
r_tree_train=roc_auc_score(y_train, y_pred_train)
r_tree_test=roc_auc_score(y_val, y_pred_val)
print(r_tree_train)
print(r_tree_test)

In [ ]:
scores = get_metrics(y_train,y_pred_train,y_val,y_pred_val,"Decision Tree_rand",scores)
scores

In [ ]:
clf_rf=RandomForestClassifier(n_estimators=50)
clf_rf.fit(X=X_train, y=y_train)
y_pred_train = clf_rf.predict(X_train)
print(roc_auc_score(y_train,y_pred_train))
y_pred_test = clf_rf.predict(X_val)
print(roc_auc_score(y_val,y_pred_test))

In [ ]:
scores = get_metrics(y_train,y_pred_train,y_val,y_pred_test,"Random_forest_BASIC",scores)
scores

In [ ]:
rfc = RandomForestClassifier(n_jobs=-1)

max_features = np.round(np.random.normal(loc=0.25, scale=0.1, size=15),2)
max_features[max_features <= 0.01] = 0.01
max_features[max_features > 1.0] = 1.0

model_params = {
    
    'n_estimators': np.random.uniform(1,80, 10).astype(int),
    # normally distributed max_features, with mean .25 stddev 0.1, bounded between 0 and 1
    'max_features': list(max_features),
    # uniform distribution from 0.01 to 0.2
    'min_samples_split': np.round(np.random.uniform(0.01, 0.2, 10),2),
    # randomly sample numbers from 4 to 15
    'max_depth': np.random.uniform(2,20, 10).astype(int)
}
rf = RandomizedSearchCV(estimator=rfc, param_distributions=model_params, n_iter=300, cv=5, scoring='roc_auc', n_jobs=-1)
rf.fit(X_train, y_train)

In [ ]:
rf.best_estimator_

In [ ]:
rf_random = rf.best_estimator_

y_pred_train = rf_random.predict(X_train)
print(roc_auc_score(y_train,y_pred_train))
y_pred_val = rf_random.predict(X_val)
print(roc_auc_score(y_val,y_pred_val))

In [ ]:
scores = get_metrics(y_train,y_pred_train,y_val,y_pred_val,"Random_forest_random",scores)
scores

In [ ]:
ada=AdaBoostClassifier()

ada.fit(X_train, y_train)
train_pred = ada.predict(X_train)
test_pred = ada.predict(X_val)

ada_train=roc_auc_score(y_train, train_pred)
ada_test=roc_auc_score(y_val, test_pred)
print(tree_train)
print(tree_test)

In [ ]:
scores = get_metrics(y_train,train_pred,y_val,test_pred ,"ada_basic",scores)
scores

In [ ]:
ada=AdaBoostClassifier(DecisionTreeClassifier(class_weight='balanced'))

ada.fit(X_train, y_train)
train_pred = ada.predict(X_train)
test_pred = ada.predict(X_val)

ada_train=roc_auc_score(y_train, train_pred)
ada_test=roc_auc_score(y_val, test_pred)
print(ada_train)
print(ada_test)

In [ ]:
scores = get_metrics(y_train,train_pred,y_val,test_pred,"Ada_Boost_wt_balenced",scores)
scores

In [ ]:
gb=GradientBoostingClassifier(n_estimators=50,learning_rate=0.01,subsample=0.1)
gb.fit(X_train, y_train)
train_pred = gb.predict(X_train)
test_pred = gb.predict(X_val)

gb_train=roc_auc_score(y_train, train_pred)
gb_test=roc_auc_score(y_val, test_pred)
print(gb_train)
print(gb_test)

In [ ]:
scores = get_metrics(y_train,train_pred,y_val,test_pred,"G_boost",scores)
scores

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
smote = SMOTE(random_state=123)
os_train_X, os_train_y = smote.fit_resample(X_train, y_train)

os_train_X = pd.DataFrame(data=os_train_X)
os_train_y= pd.DataFrame(data=os_train_y)

clf_GBM_smot = GradientBoostingClassifier(n_estimators=50,
                                       learning_rate=0.01,
                                        random_state=123)
clf_GBM_smot.fit(os_train_X, os_train_y)
y_pred_train = clf_GBM_smot.predict(os_train_X)
y_pred_val = clf_GBM_smot.predict(X_val)

In [ ]:
scores = get_metrics(os_train_y,y_pred_train,y_val,y_pred_val,"GBM_SMOTe",scores)
scores

In [ ]:
XGB_model = XGBClassifier(n_estimators=50, gamma=0.5,learning_rate=0.1)
%time XGB_model.fit(X_train, y_train)

y_pred_train = XGB_model.predict(X_train)
y_pred_test = XGB_model.predict(X_val)
print(roc_auc_score(y_train,y_pred_train))
print(roc_auc_score(y_val,y_pred_test))

In [ ]:
scores = get_metrics(y_train,y_pred_train,y_val,y_pred_test,"XG Boost",scores)
scores

In [ ]:
np.linspace(0.02,0.5,2)

In [ ]:
XGB = XGBClassifier(n_jobs=-1)
 
# Use a grid over parameters of interest
param_grid = {
     'colsample_bytree': np.linspace(0.2,0.8,1),
     'n_estimators':[10,15,18,20],
     'max_depth': [2,3,4]
}

CV_XGB = GridSearchCV(estimator=XGB, param_grid=param_grid, cv= 10)
%time CV_XGB.fit(X = X_train, y=y_train)

In [ ]:
best_xgb_model = CV_XGB.best_estimator_
print (CV_XGB.best_score_, CV_XGB.best_params_)

In [ ]:
y_pred_train=best_xgb_model.predict(X_train)
y_pred_test=best_xgb_model.predict(X_val)
print(roc_auc_score(y_train,y_pred_train))
print(roc_auc_score(y_val,y_pred_test))

In [ ]:
scores = get_metrics(y_train,y_pred_train,y_val,y_pred_test,"XGBoost_BestParameters_GCV2",scores)
scores

In [ ]:
models = [
    
        DecisionTreeClassifier(max_depth=4),
    
        RandomForestClassifier(class_weight='balanced'),
        
        GradientBoostingClassifier(),


        AdaBoostClassifier()
]

In [ ]:
models = [clf_random,rf_random,ada,clf_GBM_smot]

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
le = preprocessing.LabelEncoder()
le.fit(y_train)

In [ ]:
from vecstack import stacking

In [ ]:
y_train1 = le.transform(y_train)
y_test1 = le.transform(y_val)

In [ ]:
S_train, S_test = stacking(models,                   
                           X_train, y_train1, X_val,   
                           regression=False,  
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric=roc_auc_score, 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)
model = XGBClassifier(random_state=0, 
                           n_estimators=100, max_depth=3)
model = model.fit(S_train, y_train1)
y_pred_train=model.predict(S_train)
y_pred_test = model.predict(S_test)

In [ ]:
scores = get_metrics(y_train1,y_pred_train,y_test1,y_pred_test,"Stacking",scores)
scores

In [ ]:
test=pd.read_csv('../input/jobathon-analytics-vidhya-health-insurance/Test.csv')
test.shape

In [ ]:
sample=pd.read_csv('../input/jobathon-analytics-vidhya-health-insurance/sample_submission.csv')
sample.shape

In [ ]:
columns=['Accomodation_Type','Reco_Insurance_Type','Is_Spouse','Holding_Policy_Type',
          'Health Indicator','Holding_Policy_Duration','Reco_Policy_Cat']
for col in columns:
    test[col]=test[col].astype('category')

In [ ]:
bins=0,5,10,15,20,25
Reco_Policy_bin = ['one','two','three','four','five']
test['recommended_insurance_bin']=pd.cut(test['Reco_Policy_Cat'], bins, labels=Reco_Policy_bin)

In [ ]:
test.drop(['ID','City_Code','Region_Code','Reco_Policy_Cat'], axis=1,inplace=True)

In [ ]:
num_col = test.select_dtypes(['int64','float64']).columns
cat_col = test.select_dtypes(['category']).columns

In [ ]:
impute=SimpleImputer()
impute.fit(test[num_col])
test[num_col] = impute.transform(test[num_col])

impute = SimpleImputer(strategy='most_frequent')
impute.fit(test[cat_col])
test[cat_col] = impute.transform(test[cat_col])

In [ ]:
standard=StandardScaler()
standard.fit(test[num_col])
test_std = standard.transform(test[num_col])

test_num = pd.DataFrame(test_std, columns=num_col)

In [ ]:
one_hot = OneHotEncoder()
one_hot.fit(test[cat_col])
one_hot_col = list(one_hot.get_feature_names(cat_col))
test_ohe = one_hot.transform(test[cat_col])
test_cat = pd.DataFrame(test_ohe.todense(), columns=one_hot_col)

test_final = pd.concat([test_num,test_cat],axis=1)
test_final.head(2)

In [ ]:
test.head(2)

In [ ]:
prediction = clf_random.predict(test_final)
predictions = pd.DataFrame(prediction)

In [ ]:
predictions[0]

In [ ]:
sample.head(2)

In [ ]:
sample.drop('Response',axis=1,inplace=True)

In [ ]:
sample['Response']=predictions[0]
sample.head(5)

In [ ]:
sample.Response.value_counts()

In [ ]:
sample.to_csv('my_FINAL_submission.csv', index=False)
print("Your submission was successfully saved!")